In [1]:
%matplotlib inline

/Users/shoji/go/src/github.com/mikanfactory/CanDataAnalyzer/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import json
import math
import pprint
from sklearn.mixture import BayesianGaussianMixture
from skbayes.mixture_models import VBBMM

ImportError: No module named skbayes.mixture_models

In [ ]:
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
target = "../data/input/R5_L53E-DM05_2119_20161102142424_comb.csv"
df = pd.read_csv(target)

In [ ]:
len(df)

In [4]:
accel = "RawPedal_APOFS[%]"
brake = "BRKSWTM"
speed= "VSO[km/h]"

In [5]:
c0 = "Curve_0[\xef\xbf\xbd\xef\xbf\xbdm]"
c1 = "Curve_1[\xef\xbf\xbd\xef\xbf\xbdm]"
c2 = "Curve_2[\xef\xbf\xbd\xef\xbf\xbdm]"
c3 = "Curve_3[\xef\xbf\xbd\xef\xbf\xbdm]"
c4 = "Curve_4[\xef\xbf\xbd\xef\xbf\xbdm]"
c5 = "Curve_5[\xef\xbf\xbd\xef\xbf\xbdm]"
c6 = "Curve_6[\xef\xbf\xbd\xef\xbf\xbdm]"

In [6]:
RoadTypes = ["都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道",  "一般道1、一般道2、一般道3", "その他"]
HighSpeeds = ["都市間高速", "都市高速", "有料道路"]

In [7]:
df[df["RoadType"].apply(lambda x: math.isnan(x))].index

Int64Index([], dtype='int64')

# Edit Average Velocity

In [8]:
avgVelocity = "AverageVelocity"

In [9]:
df[avgVelocity] = 0

In [12]:
accelOffs = df[df[accel] == -1].index

In [18]:
starts = df.ix[accelOffs].index.map(lambda x: max(0, x-1000))

In [20]:
for start, stop in zip(starts, accelOffs):
    df.loc[stop, avgVelocity] = df.ix[start:stop][speed].mean()

In [21]:
df[df[accel] == -1].describe()

,Time,YEAR[year],SEC[sec],MONTH[month],MINUTE[min],HOUR[hour],DAY[day],STAT,GPS_S_NUM,LON_GPS[deg],...,Jerk,slope,THW,TTC,RF,DistSTOP,AverageVelocity,MaxSpeed,MinSpeed,CurveAverage
count,215.000000,215.0,215.000000,215.0,215.000000,215.000000,215.0,215.0,215.000000,215.000000,...,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.00000,215.0,215.000000
mean,3293.651163,16.0,26.660465,11.0,29.674419,5.683721,2.0,0.0,8.097674,139.398241,...,0.266150,-1.290068,1.778668,33.110028,0.162352,1536.705049,25.198618,77.50214,0.0,184.330233
std,1867.991490,0.0,16.930343,0.0,17.720996,0.582024,0.0,0.0,2.467498,0.051107,...,0.949315,5.601274,1.207226,838.985706,1.452627,3973.608338,18.786103,24.48561,0.0,70.973918
min,46.600000,16.0,0.000000,11.0,0.000000,5.000000,2.0,0.0,0.000000,139.314082,...,-4.111111,-66.623377,0.000000,-4063.129412,-17.384074,3.551944,3.198615,16.31000,0.0,41.000000
25%,1788.750000,16.0,12.000000,11.0,15.500000,5.000000,2.0,0.0,8.000000,139.361563,...,-0.194444,-3.205183,1.044687,-36.594932,0.000000,109.048889,12.367428,51.41000,0.0,117.500000
50%,3177.600000,16.0,26.000000,11.0,32.000000,6.000000,2.0,0.0,9.000000,139.382225,...,0.333333,-0.418702,1.883270,-11.891205,0.335316,333.285833,22.132208,76.96000,0.0,209.000000
75%,4935.250000,16.0,40.000000,11.0,46.000000,6.000000,2.0,0.0,9.000000,139.462282,...,0.888889,0.833205,2.462282,0.000000,0.472840,759.413333,31.252498,106.19000,0.0,251.000000
max,6612.900000,16.0,59.000000,11.0,59.000000,7.000000,2.0,0.0,11.000000,139.484502,...,3.111111,8.924843,6.141965,9952.517647,4.787983,17293.401110,100.905365,106.19000,0.0,255.000000


# Edit Curve Radius

In [14]:
curveAve = "CurveAverage"

In [15]:
df[curveAve] = 0

In [16]:
highSpeeds = df["RoadType"].apply(lambda x: x in [0.0, 2.0, 3.0])

In [17]:
df.loc[highSpeeds, curveAve] = df[highSpeeds].apply(lambda x: min(x[c0], x[c1], x[c2], x[c3], x[c4], x[c5], x[c6]), axis=1)

In [18]:
lowSpeeds = df["RoadType"].apply(lambda x: x not in [0.0, 2.0, 3.0])

In [19]:
df.loc[lowSpeeds, curveAve] = df[lowSpeeds].apply(lambda x: min(x[c0], x[c1], x[c2], x[c3], x[c4]), axis=1)

# Update csv

In [25]:
integers = ['BRKSWTM', 'RawPedal_APOFS[%]', 'D_BRANCH_FLG', 'RoadType', 'VC_GreenLamp', 'VC_RedLamp', 'VC_RightLamp', 'VC_UpLamp', 'VC_LeftLamp', 'VC_StopSign', 'VC_30Sign', 'VC_50Sign', 'VC_CarBrake', 'VC_LeftCar', 'VC_ManCycleCount', 'VC_ManCycle', 'VC_LaneCount.1']

In [45]:
df["RoadType"] = df["RoadType"].fillna(-1)

In [46]:
df["D_BRANCH_FLG"] = df["D_BRANCH_FLG"].fillna(-1)

In [48]:
df[integers] = df[integers].astype(int)